In [1]:
import wget
import os
import pandas as pd
import xlsxwriter

En primer lugar accedemos a las urls con las listas de sancionados de EEUU en formato .csv y .xml y las guardamos en local. Cada vez que este proceso es ejecutado, se borran los archivos antiguos y se vuelven a cargar en nuestro directorio local.

In [2]:
urls = ['https://www.treasury.gov/ofac/downloads/sdn.csv',
        'https://www.treasury.gov/ofac/downloads/add.csv',
        'https://www.treasury.gov/ofac/downloads/alt.csv',
        'https://www.treasury.gov/ofac/downloads/sdn.xml']

path = 'UE_EEUU_data_crudo'

for url in urls:
    filename = path + '/' + os.path.basename(url) # get the full path of the file
    if os.path.exists(filename):
        os.remove(filename) # if exist, remove it directly
    wget.download(url, out=filename) # download it to the specific path.

A continuación cargamos los .csv con la información de sancionados de EEUU en tres DataFrames relacionados por la columna uid_pk_sdn, que es la clave primaria del DataFrame df_sdn y la foránea del resto.

In [3]:
df_sdn = pd.read_csv('UE_EEUU_data_crudo/sdn.csv',usecols=list(range(3)),header=None).iloc[0:10409]
df_sdn.columns = ['uid_pk_sdn','LastName','sdnType']
df_sdn.head()

,uid_pk_sdn,LastName,sdnType
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-


In [4]:
df_add = pd.read_csv('UE_EEUU_data_crudo/add.csv',header=None,usecols=list(range(5))).iloc[0:15187]
df_add.columns = ['uid_fk_sdn','uid_pk_add','Address','City','Country']
df_add.head()

,uid_fk_sdn,uid_pk_add,Address,City,Country
0,36,25.0,-0-,Havana,Cuba
1,173,129.0,"Ibex House, The Minories",London EC3N 1DY,United Kingdom
2,306,199.0,Zweierstrasse 35,Zurich CH-8022,Switzerland
3,306,200.0,Avenida de Concha Espina 8,Madrid E-28036,Spain
4,306,201.0,"Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-c...",Tokyo 103,Japan


In [5]:
df_alt = pd.read_csv('UE_EEUU_data_crudo/alt.csv',header=None,usecols=list(range(4))).iloc[0:14099]
df_alt.columns = ['uid_fk_sdn','uid_pk_alt','AliasType','Alias']
df_alt.head()

,uid_fk_sdn,uid_pk_alt,AliasType,Alias
0,36,12.0,aka,AERO-CARIBBEAN
1,173,57.0,aka,AVIA IMPORT
2,306,220.0,aka,NATIONAL BANK OF CUBA
3,540,471.0,aka,COIBA
4,552,475.0,aka,CRYMSA


Para las listas de la UE, la petición a la URL se hace con un token. No sirve el código utilizado anteriormente porque no acepta el método wget.download. Este proceso no almacena la tabla en local, si no que genera directamente en Python un archivo con los datos. 

In [6]:
import csv
import urllib.request
from io import StringIO

url = 'https://webgate.ec.europa.eu/fsd/fsf/public/files/csvFullSanctionsList_1_1/content?token=n008rnnv'
respuesta = urllib.request.urlopen(url)
f = StringIO(bytearray(respuesta.read()).decode())
archivo = csv.reader(f)

A continuación transformamos el archivo generado a DataFrame.

In [7]:
df = pd.DataFrame(archivo)
df.rename(columns={0:'column'},inplace=True)
df = df[['column']]
df = df.column.str.split(pat=';',expand=True)

Pasamos los datos en crudo de la lista de la UE a local desde el Dataframe sin depurar. Cada vez que se ejecute el proceso, se elimina el .xlsx con los datos en crudo antiguos y se cargan los nuevos datos en crudo.

In [8]:
filename = path + '/' + 'sancionlist_UE.csv' # get the full path of the file
if os.path.exists(filename):
    os.remove(filename) # if exist, remove it directly

df.to_csv('UE_EEUU_data_crudo/sancionlist_UE.csv')

Depuramos el DataFrame para poner nombre a las columnas y quedarnos solo con las columnas que nos interesan.

In [9]:
df_ue = df.iloc[1:,[0,8,19,21,34,35,38,39,43]].rename(columns={0:'GenerationDate',8:'SanctionType',19:'NameAlias',
                                                     21:'Gender',34:'AddressCity',35:'AddressStreet',
                                                     38:'AddressRegion',39:'AddressPlace',43:'AddressCountry'})

df_ue.head()

,GenerationDate,SanctionType,NameAlias,Gender,AddressCity,AddressStreet,AddressRegion,AddressPlace,AddressCountry
1,26/07/2022,person,Saddam Hussein Al-Tikriti,M,,,,,
2,26/07/2022,person,Abu Ali,,,,,,
3,26/07/2022,person,Abou Ali,,,,,,
4,26/07/2022,person,,,,,,,
5,26/07/2022,person,,,,,,,


Finalmente exportamos todos los DataFrame depurados en formato .xlsx a la carpeta local 'UE_EEUU_data_depurada'. Cada vez que se ejecute el proceso, se eliminan los sheets con los datos depurados antiguos y se le cargan los nuevos.

In [10]:
dfs_xlsx = ['df_sdn.xlsx','df_add.xlsx','df_alt.xlsx','df_ue.xlsx']

path = 'UE_EEUU_data_depurada'


filename = path + '/' + 'sanctionlist.xlsx' # get the full path of the file

if os.path.exists(filename):
    os.remove(filename) # if exist, remove it directly

with pd.ExcelWriter('UE_EEUU_data_depurada/sanctionlist.xlsx', engine='xlsxwriter') as writer:
    
    df_sdn.to_excel(writer, sheet_name='EEUU Name & Sanction Type')
    df_add.to_excel(writer, sheet_name='EEUU Location Information')
    df_alt.to_excel(writer, sheet_name='EEUU Alias Information')
    df_ue.to_excel(writer, sheet_name='UE All Information')
